# SIMP Topology Optimization using Gridap in Julia

UQLID Lab at NAU, 2023 

Rahaman Lab at IIT Bhubaneswar 

In the following code: 

$p$ = densities 

$pf$ = filtered densities

$pth$ = thresholded densities 

### Load required packages 

In [1]:
# FEA 
using  Gridap
using  Gridap.Geometry
using  Gridap.Fields
using  Gridap.TensorValues 
using  Gridap.CellData 

# Meshing 
using  Gmsh 
using  GridapGmsh

using  LinearAlgebra 

# Gardient calculation 
using  ChainRulesCore, Zygote
import ChainRulesCore: rrule 

# For MMA 
using  NLopt 

# For plotting 
using CairoMakie, GridapMakie 

ArgumentError: ArgumentError: Package ChainRulesCore not found in current path.
- Run `import Pkg; Pkg.add("ChainRulesCore")` to install the ChainRulesCore package.

### Constant parameters 
Material properties, such as modulus of elasticity and Poisson's ratio. 
The SIMP penalty is generally assumed as 3~5. 

In [ ]:
const E_mat = 1.0
const ν_mat = 0.3 
const penal = 3

### Create the mesh or load already created mesh file using gmsh 
A GUI is also available to use gmsh 

In [ ]:
# ################################################
# # Create the mesh 
# ################################################

# const L = 60
# const H = 20
# const Hp = H/2
# const hf = L/200
# const Hw = H/20

# gmsh.initialize()
# gmsh.option.setNumber("General.Terminal", 1)
# #gmsh.option.setNumber("Mesh.Algorithm", 8) 
# #gmsh.option.setNumber("Mesh.RecombineAll", 1)

# gmsh.model.geo.addPoint(0.0, 0.0 , 0.0, hf ,1)
# gmsh.model.geo.addPoint(L, 0.0, 0.0, hf, 2)
# gmsh.model.geo.addPoint(L, Hp-Hw, 0.0, hf, 21)
# gmsh.model.geo.addPoint(L, Hp+Hw, 0.0, hf, 22)
# gmsh.model.geo.addPoint(L, H, 0.0, hf, 3)
# gmsh.model.geo.addPoint(0.0, H, 0.0, hf, 4)

# gmsh.model.geo.addLine(1, 2, 1)
# gmsh.model.geo.addLine(2, 21, 2)
# gmsh.model.geo.addLine(21, 22, 21)
# gmsh.model.geo.addLine(22, 3, 22)
# gmsh.model.geo.addLine(3, 4, 3)
# gmsh.model.geo.addLine(4, 1, 4)

# gmsh.model.geo.addCurveLoop([1,2,21,22,3,4],1)
# gmsh.model.geo.addPlaneSurface([1], 1)
# gmsh.model.addPhysicalGroup(2, [1],1)

# gmsh.model.addPhysicalGroup(1, [21],1)
# gmsh.model.addPhysicalGroup(1, [4],2)

# gmsh.model.setPhysicalName(2, 1, "Domain")

# gmsh.model.setPhysicalName(1, 1, "LoadLine")
# gmsh.model.setPhysicalName(1, 2, "LeftSupport")

# gmsh.model.geo.synchronize()
# gmsh.model.mesh.generate(2)
# gmsh.write("mesh.msh")
# gmsh.finalize()

#################################################
# Load the mesh 
#################################################
model = GmshDiscreteModel("mesh.msh"); 

#################################################
# Save the mesh in a vtk file 
#################################################
writevtk(model ,"Cantilever"); 

Info    : Reading 'Cantilever.msh'...
Info    : 13 entities
Info    : 15863 nodes
Info    : 31264 elements
Info    : Done reading 'Cantilever.msh'


### Constitutive matrix (1 for plane stress, 2 for plane strain) 
Elasticity tensor, constitutive relation, and SIMP penalty relation are formulated here. 

In [5]:
function  ElasFourthOrderConstTensor(E,ν,PlanarState)
    # 1 for  Plane  Stress  and 2 Plane  Strain  Condition
    if  PlanarState  == 1
        C1111 =E/(1-ν*ν)
        C1122 = (ν*E)/(1-ν*ν)
        C1112 = 0.0
        C2222 =E/(1-ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    elseif  PlanarState  == 2
        C1111 = (E*(1-ν*ν))/((1+ν)*(1-ν-2*ν*ν))
        C1122 = (ν*E)/(1-ν-2*ν*ν)
        C1112 = 0.0
        C2222 = (E*(1-ν))/(1-ν-2*ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    end
    C_ten = SymFourthOrderTensorValue(C1111 ,C1112 ,C1122 ,C1112 ,C1212 ,C2212 ,C1122 ,C2212 ,C2222)
    return   C_ten
end 

function σfun(ε)
    σ = C_mat⊙ε
    return  σ
end 

function Em(p)
    Em = p ^ penal
    return Em
end 

const  C_mat = ElasFourthOrderConstTensor(E_mat ,ν_mat ,1);

### Finite Element Analysis 

In [7]:
order = 1
reffe_Disp = ReferenceFE(lagrangian ,VectorValue{2,Float64},order)
V0_Disp = TestFESpace(model,reffe_Disp;conformity =:H1,
    dirichlet_tags = ["LeftSupport"],
    dirichlet_masks =[(true,true)])
uh = zero(V0_Disp)
U0_Disp = V0_Disp 

degree = 2*order
Ω= Triangulation(model)
dΩ= Measure(Ω,degree) 

labels = get_face_labeling(model)
LoadTagId = get_tag_from_name(labels ,"LoadLine")
Γ_Load = BoundaryTriangulation(model ;tags = LoadTagId)
dΓ_Load = Measure(Γ_Load ,degree)
n_Γ_Load = get_normal_vector(Γ_Load) 

p_reffe = ReferenceFE(lagrangian, Float64, 0)
Q = TestFESpace(Ω, p_reffe, vector_type = Vector{Float64})
P = Q
np = num_free_dofs(P) 

pf_reffe = ReferenceFE(lagrangian, Float64, 1)
Qf = TestFESpace(Ω, pf_reffe, vector_type = Vector{Float64})
Pf = Qf 

fem_params = (;V0_Disp, U0_Disp, Q, P, Qf, Pf, np, Ω, dΩ, n_Γ_Load, dΓ_Load) 

A_Disp(u,v,pth) =  ((p->Em(p))∘pth) * (ε(v) ⊙ (σfun∘(ε(u))))
function MatrixA(pth; fem_params)
    A_mat = assemble_matrix(fem_params.U0_Disp, fem_params.V0_Disp) do u, v
        ∫(A_Disp(u,v,pth))fem_params.dΩ
    end
    return lu(A_mat)
end

f = VectorValue(0,-1.0) # Load vector 

function  stepDisp(fem_params,pth)
    A_Disp(u,v,pth) = ((p->Em(p))∘pth) * ε(v) ⊙ (σfun∘(ε(u)))
    a_Disp(u,v) = ∫(A_Disp(u,v,pth))fem_params.dΩ
    b_Disp(v) = ∫(v ⋅ f)fem_params.dΓ_Load
    op_Disp = AffineFEOperator(a_Disp ,b_Disp ,fem_params.U0_Disp ,fem_params.V0_Disp)
    uh_out = solve(op_Disp)
    return  get_free_dof_values(uh_out)
end 

function MatrixOf(fem_params)

    return assemble_matrix(fem_params.U0_Disp, fem_params.V0_Disp) do u, v
             0.5*∫((∇(u))' ⊙ (C_mat ⊙ ∇(v)))fem_params.dΩ
    end
end
@show size(MatrixOf(fem_params)); 

size(MatrixOf(fem_params)) = (31594, 31594)


### Filtering and thresholding 
Helmholtz PDE-based filter is used here. 

(see: Lazarov, Boyan Stefanov, and Ole Sigmund. "Filters in topology optimization based on Helmholtz‐type differential equations." International Journal for Numerical Methods in Engineering 86.6 (2011): 765-781.) 

A smooth approximation of the Heaviside step function is used as the thresholding function. $\beta$ determines the strength of the approximation, and $\eta$ determines the location. Note that $\eta=0.5$ is standard in TO. 

In [8]:
elemsize = 0.3
r = (3*elemsize)/(2*sqrt(3)) #0.025           # Filter radius
β = 4                       # β∈[1,∞], threshold sharpness
η = 0.5                     # η∈[0,1], threshold center

a_f(r, u, v) = r^2 * (∇(v) ⋅ ∇(u))

function Filter(p0; r, fem_params)
    ph = FEFunction(fem_params.P, p0)
    op = AffineFEOperator(fem_params.Pf, fem_params.Qf) do u, v
        ∫(a_f(r, u, v))fem_params.dΩ + ∫(v * u)fem_params.dΩ, ∫(v * ph)fem_params.dΩ
      end
    pfh = solve(op)
    return get_free_dof_values(pfh)
end

function Threshold(pfh; β, η)
    return  ((tanh(β * η) + tanh(β * (pfh - η))) / (tanh(β * η) + tanh(β * (1.0 - η)))) 

end 

NO_FIELDS = ZeroTangent() 

Dptdpf(pf, β, η) = β * (1.0 - tanh(β * (pf - η))^2) / (tanh(β * η) + tanh(β * (1.0 - η))) # Gradient of thresholding function 
DEdpf(pf, β, η)= penal * ((Threshold(pf; β, η)) ^ (penal-1)) * Dptdpf(pf, β, η) # Gradient of density^penal
DAdpf(u, v, pfh; β, η) = ((p->DEdpf(p, β, η)) ∘ pfh) * (ε(v) ⊙ (σfun∘(ε(u)))); 

### FEA result with the entire domain filled with density 1 elements 
For checking! 

In [10]:
# Comment/Uncomment for FEA Analysis 
p0 = ones(fem_params.np)
pf_vec = Filter(p0;r, fem_params)
pfh = FEFunction(fem_params.Pf, pf_vec)
pth = (pf -> Threshold(pf; β, η)) ∘ pfh
A_mat = MatrixA(pth; fem_params)
u_vec = stepDisp(fem_params,pth)
uh = FEFunction(fem_params.U0_Disp, u_vec)

writevtk(Ω,"FEAresultsmbb",cellfields=["uh"=>uh,"p0"=>p0,"pth"=>pth]);

### Gradient calculation 
Using adjoint method 

In [11]:
function gf_pf(pf_vec; β, η, fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    u_vec = stepDisp(fem_params,pth)
    K_mat = MatrixOf(fem_params)
    u_vec' * K_mat * u_vec
end 

function rrule(::typeof(gf_pf), pf_vec; β, η, fem_params)
    function U_Disp_pullback(dg)
      NO_FIELDS, dg * Dgfdpf(pf_vec; β, η, fem_params) 
    end
    gf_pf(pf_vec; β, η, fem_params), U_Disp_pullback
end

function Dgfdpf(pf_vec; β, η, fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    A_mat = MatrixA(pth; fem_params)
    u_vec = stepDisp(fem_params,pth)
    O_mat = MatrixOf(fem_params)
    uh = FEFunction(fem_params.U0_Disp, u_vec)
    w_vec =  A_mat' \ (O_mat * u_vec)
    wconjh = FEFunction(fem_params.U0_Disp, w_vec)
    l_temp(dp) = ∫(-2*DAdpf(wconjh,uh, pfh; β, η) * dp)fem_params.dΩ
    dgfdpf = assemble_vector(l_temp, fem_params.Pf)

    return dgfdpf
end 

function pf_p0(p0; r, fem_params)
    pf_vec = Filter(p0; r, fem_params)
    pf_vec
end

function rrule(::typeof(pf_p0), p0; r, fem_params)
  function pf_pullback(dgdpf)
    NO_FIELDS, Dgdp(dgdpf; r, fem_params)
  end
  pf_p0(p0; r, fem_params), pf_pullback
end

function Dgdp(dgdpf; r, fem_params)
    Af = assemble_matrix(fem_params.Pf, fem_params.Qf) do u, v
        ∫(a_f(r, u, v))fem_params.dΩ + ∫(v * u)fem_params.dΩ
    end
    wvec = Af' \ dgdpf
    wh = FEFunction(fem_params.Pf, wvec)
    l_temp(dp) = ∫(wh * dp)fem_params.dΩ
    return assemble_vector(l_temp, fem_params.P)
end 

function gf_p(p0::Vector; r, β, η, fem_params)
    pf_vec = pf_p0(p0; r, fem_params)
    gf_pf(pf_vec; β, η, fem_params)
end

function gf_p(p0::Vector, grad::Vector; r, β, η, fem_params)
    if length(grad) > 0
        dgdp, = Zygote.gradient(p -> gf_p(p; r, β, η, fem_params), p0)
        grad[:] = dgdp
    end
    gvalue = gf_p(p0::Vector; r, β, η, fem_params)
    open("gvaluemma.txt", "a") do io
        write(io, "$gvalue \n")
    end
    gvalue
end; 

### Volume constraint calculation 
Volume should be estimated with filtered densities and not with thresholded densities. 

In [12]:
volfrac = 0.3 

dv=get_array(∫(1)fem_params.dΩ)
getpoints = get_cell_points(Ω)

gradc = zeros(fem_params.np)
function cf_p(p0::Vector, gradc::Vector; r, β, η, fem_params)
    if length(gradc) > 0
        gradc[:] = dv
    end
    pf_vec = pf_p0(p0; r, fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    pthxtr = evaluate(pfh⊙dv,getpoints)
    return sum(pthxtr)[1] - volfrac*sum(dv)
end; 

### MMA to optimize 
Svanberg, Krister. "The method of moving asymptotes—a new method for structural optimization." International journal for numerical methods in engineering 24.2 (1987): 359-373. 

In [13]:
function gf_p_optimize(p_init; r, β, η, TOL=1e-4, MAX_ITER, fem_params)
    ##################### Optimize #################
    opt = Opt(:LD_MMA, fem_params.np)
    opt.lower_bounds = 0.001
    opt.upper_bounds = 1
    opt.xtol_rel = TOL
    opt.maxeval = MAX_ITER
    opt.min_objective = (p0, grad) -> gf_p(p0, grad; r, β, η, fem_params)
    inequality_constraint!(opt, (p0, gradc) -> cf_p(p0, gradc; r, β, η, fem_params), 1e-8)
    (g_opt, p_opt, ret) = optimize(opt, p_init)
    @show numevals = opt.numevals # the number of function evaluations
    println("got $g_opt at $p_opt after $numevals iterations (returned $ret)")
    return g_opt, p_opt

end; 

### Run the MMA 

In [14]:
# p0 = rand(fem_params.np) # For randomized densities 
grad = zeros(fem_params.np)

p_opt = fill(volfrac, fem_params.np)   # Uniform Initial guess
g_opt = 0

TOL = 1e-5 # tolerance of the MMA 
MAX_ITER = 5000 # Maximum iteration constraints 

g_opt, p_opt = gf_p_optimize(p_opt; r, β, η, TOL, MAX_ITER, fem_params);

###############################################
# For a list of β in the thresholding function 
#----------------------------------------------
# β_list = [4.0,8.0,16.0] 
# for bi = 1 : 3
#     β = β_list[bi]
#     g_opt, p_temp_opt = gf_p_optimize(p_opt; r, β, η, fem_params)
#     global p_opt = p_temp_opt
# end 
# @show g_opt 

numevals = opt.numevals = 4000


got 2728.68008993637 at [0.001, 0.0015898983641424009, 0.0010897510357913078, 0.001, 0.9192254877050856, 1.0, 1.0, 0.8232156468839041, 0.9517091478769731, 0.0010007226888499043, 1.0, 0.0014255654584740959, 1.0, 0.8023364408741392, 0.0010000054532377266, 0.969048821248712, 0.001510532771128251, 0.0010067735260344006, 0.7562039912880514, 0.8859540953079423, 0.0010146049395848256, 1.0, 0.5680907988646732, 0.001, 0.001, 0.001, 1.0, 0.001, 0.8612970735360784, 0.9925169182942218, 0.9525751769553421, 1.0, 1.0, 1.0, 0.9905156583569411, 0.001052575920973846, 0.7288346181938061, 0.9363251082632895, 0.001, 0.001, 0.0033979781175753265, 0.9751558044752712, 0.001, 0.001070146949921914, 1.0, 0.9373948154731205, 1.0, 0.7866082632165298, 0.0014923619204990795, 0.0010033617205992904, 0.001, 1.0, 1.0, 0.001, 0.001, 0.5311094865322395, 1.0, 0.001, 1.0, 0.006366879022143611, 0.0010134369808085545, 0.0026571884533078825, 0.0012192732100212743, 0.0012737054230122496, 0.001030214748735821, 1.0, 0.001, 0.8652

### Postprocessing 
Redefine the thresholding function with a sharp increase (i.e., increase β) to get a crisp boundary 

In [15]:
βpost=64
function Thresholdp(pfh; βpost, η)
    return ((tanh(βpost * η) + tanh(βpost * (pfh - η))) / (tanh(βpost * η) + tanh(βpost * (1.0 - η)))) 
end 

pf_vec = pf_p0(p_opt; r, fem_params)
pfh = FEFunction(fem_params.Pf, pf_vec)
pth = (pf -> Thresholdp(pf; βpost, η)) ∘ pfh; 

### Save the optimized design in png and vtk files 

In [ ]:
fig, ax, plt = plot(fem_params.Ω, pth, colormap = :binary)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(3)
ax.title = "Optimized Design"
# rplot = 110 # Region for plot
limits!(ax, 0, 60, 0, 20)
save("result_image.png", fig)

writevtk(Ω,"result",cellfields= ["p_opt"=>p_opt,"pfh"=>pfh,"pth"=>pth]);